# Luminar

## Minimal Code

In [1]:
from typing import Iterable, NamedTuple

import torch
from datasets import load_dataset
from torch import nn
from torch.utils.data import DataLoader
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
)

In [11]:
ds = load_dataset("liamdugan/raid", "raid_test")
ds

test.csv:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'generation'],
        num_rows: 672000
    })
})

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
        num_rows: 5615820
    })
    extra: Dataset({
        features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
        num_rows: 2039100
    })
})

In [6]:
ds.filter(lambda x: x["model"] == "human")

Filter:   0%|          | 0/5615820 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'adv_source_id', 'source_id', 'model', 'decoding', 'repetition_penalty', 'attack', 'domain', 'title', 'prompt', 'generation'],
    num_rows: 160452
})

### Luminar Encoder

1. Pre-Process Inputs: tokenize and pass through LLM, recording hidden states
2. Calculate _Intermediate Likelihoods_: pass each hidden state through the models LM head

In [2]:
class LuminarEncoder:
    def __init__(
        self,
        feature_dim: int = 256,
        model_name_or_path: str = "gpt2",
        device: str = ("cuda" if torch.cuda.is_available() else "cpu"),
        key: str = "text",
    ):
        self.feature_dim = feature_dim
        self.device = torch.device(device)
        self._key = key

        self.tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path
        )
        if not hasattr(self.tokenizer, "pad_token") or self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.pad_token_id = self.tokenizer.pad_token_id

        self.model: PreTrainedModel = AutoModelForCausalLM.from_pretrained(
            model_name_or_path
        )
        self.model = self.model.to(self.device)

        if hasattr(self.model, "lm_head"):
            self.model_lm_head: nn.Linear = self.model.lm_head
        elif hasattr(self.model.model, "lm_head"):
            self.model_lm_head: nn.Linear = self.model.model.lm_head
        else:
            raise ValueError("Could not find lm_head in model")

    def __call__(
        self, batch: dict[str, list[str]], /, key=None
    ) -> dict[str, list[torch.Tensor]]:
        return {"features": self.process(batch[key or self._key])}

    def process(self, batch: list[str]) -> list[torch.Tensor]:
        encoding = self.tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=self.feature_dim,
            return_tensors="pt",
        )
        batch_hidden_states = self.forward(encoding.input_ids, encoding.attention_mask)

        intermediate_likelihoods = []
        for input_ids, hidden_states in zip(encoding.input_ids, batch_hidden_states):
            intermediate_likelihoods.append(
                self.compute_intermediate_likelihoods(input_ids, hidden_states)
            )

        return intermediate_likelihoods

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
    ) -> Iterable[tuple[torch.Tensor, ...]]:
        with torch.no_grad():
            outputs = self.model(
                input_ids=input_ids.to(self.device),
                attention_mask=attention_mask.to(self.device),
                output_hidden_states=True,
            )

            # unpack hidden states to get one list of tensors per input sequence,
            # instead of one hidden state per layer in the model
            hidden_states = zip(*[hs.cpu() for hs in outputs.hidden_states])  # type: ignore

            del outputs
        return hidden_states

    def compute_intermediate_likelihoods(
        self,
        input_ids: torch.Tensor,
        hidden_states: tuple[torch.Tensor],
    ) -> torch.Tensor:
        labels = input_ids[1:].view(-1, 1)

        seq_length = min(labels.ne(self.pad_token_id).sum(), self.feature_dim)
        labels = labels[:seq_length].to(self.device)

        intermediate_likelihoods = []
        with torch.no_grad():
            for hs in hidden_states:
                hs: torch.Tensor = hs[:seq_length].to(self.device)
                il = (
                    # get layer logits
                    self.model_lm_head(hs)
                    # calculate likelihoods
                    .softmax(-1)
                    # get likelihoods of input tokens
                    .gather(-1, labels)
                    .squeeze(-1)
                    .cpu()
                )
                del hs

                # pad with zeros if sequence is shorter than required feature_dim
                if seq_length < self.feature_dim:
                    il = torch.cat([il, torch.zeros(self.feature_dim - seq_length)])

                intermediate_likelihoods.append(il)
        # stack intermediate likelihoods to get tensor of shape (feature_dim, num_layers)
        return torch.stack(intermediate_likelihoods, dim=1)

### Luminar Classifier

CNN-based classifier using _Intermediate Likelihoods_ as input features.
Here, we utilize these inherently 2D values (`seq_len * num_layers`) as 1D inputs where the second dimension is treated as input channels.

In [3]:
class ConvolutionalLayerSpec(NamedTuple):
    channels: int
    kernel_size: int | tuple[int, int]
    stride: int = 1

    @property
    def kernel_size_1d(self):
        if isinstance(self.kernel_size, int):
            return self.kernel_size
        return self.kernel_size[0]

    @property
    def kernel_size_2d(self):
        if isinstance(self.kernel_size, int):
            return (self.kernel_size, self.kernel_size)
        return self.kernel_size

    @property
    def padding(self) -> int:
        return (self.kernel_size_1d - 1) // 2

    def __repr__(self):
        return repr(tuple(self))


DEFAULT_CONV_LAYER_SHAPES = ((64, 5), (128, 3), (128, 3), (128, 3), (64, 3))


class LuminarClassifier(nn.Module):
    def __init__(
        self,
        conv_layer_shapes: Iterable[ConvolutionalLayerSpec] = DEFAULT_CONV_LAYER_SHAPES,
        projection_dim: int | None = None,
    ):
        super().__init__()
        self.conv_layers = nn.Sequential()
        for conv in conv_layer_shapes:
            conv = ConvolutionalLayerSpec(*conv)
            self.conv_layers.append(
                nn.LazyConv1d(
                    conv.channels,
                    conv.kernel_size,  # type: ignore
                    conv.stride,
                    conv.padding,
                ),
            )
            self.conv_layers.append(
                nn.LeakyReLU(),
            )
        self.conv_layers.append(nn.Flatten())

        if projection_dim:
            self.projection = nn.Sequential(
                nn.LazyLinear(projection_dim), nn.LeakyReLU()
            )
        else:
            self.projection = nn.Identity()

        self.classifier = nn.LazyLinear(1)

    def forward(self, features: torch.Tensor):
        # We are using 2D features (so `features` is a 3D tensor)
        # but we want to treat the second feature dimension as channels.
        # Thus, we need to transpose the tensor here
        features = features.transpose(1, 2)

        for layer in self.conv_layers:
            features = layer(features)

        return self.classifier(self.projection(features.flatten(1)))


## Example

### Prepare Data

In [1]:
import ray

ray.init(address="auto")

2025-05-14 11:13:56,048	INFO worker.py:1694 -- Connecting to existing Ray cluster at address: 141.2.108.212:6379...
2025-05-14 11:13:56,058	INFO worker.py:1879 -- Connected to Ray cluster. View the dashboard at http://141.2.108.212:8265 


Python version:,3.12.7
Ray version:,2.46.0
Dashboard:,http://141.2.108.212:8265


(raylet) Spilled 11379 MiB, 2 objects, write throughput 2299 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


(raylet) The autoscaler failed with the following error:
Terminated with signal 15
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/autoscaler/_private/monitor.py", line 748, in <module>
    monitor.run()
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/autoscaler/_private/monitor.py", line 605, in run
    self._run()
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/autoscaler/_private/monitor.py", line 459, in _run
    time.sleep(AUTOSCALER_UPDATE_INTERVAL_S)



In [4]:
def load_ray_dataset():
    return ray.data.from_huggingface(load_dataset("liamdugan/raid", split="train"))


ray_ds = load_ray_dataset()

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
ray_ds.write_parquet("local:///nvme/projects/PrismAI/PrismAI/data/liamdugan_raid_train/")

2025-05-14 11:42:08,993	INFO logging.py:290 -- Registered dataset logger for dataset dataset_10_0
2025-05-14 11:42:08,995	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_10_0. Full logs are in /tmp/ray/session_2025-05-14_10-45-36_614083_46283/logs/ray-data
2025-05-14 11:42:08,996	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_10_0: InputDataBuffer[Input] -> TaskPoolMapOperator[Write]


Running 0: 0.00 row [00:00, ? row/s]

- Write 1: 0.00 row [00:00, ? row/s]

2025-05-14 11:42:09,027	ERROR streaming_executor_state.py:489 -- An exception was raised from a task of operator "Write". Dataset execution will now abort. To ignore this exception and continue, set DataContext.max_errored_blocks.
Traceback (most recent call last):
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/streaming_executor_state.py", line 457, in process_completed_tasks
    bytes_read = task.on_data_ready(
                 ^^^^^^^^^^^^^^^^^^^
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/interfaces/physical_operator.py", line 137, in on_data_ready
    raise ex from None
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/interfaces/physical_operator.py", line 133, in on_data_ready
    ray.get(block_ref)
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 21, 

TaskUnschedulableError: The task is not schedulable: The node specified via NodeAffinitySchedulingStrategy doesn't exist any more or is infeasible, and soft=False was specified. task_id=39132ad0276b05af6bf564f1c746b1a7d2cc62a30b000000, task_name=Write

[2025-05-14 11:50:02,631 E 61783 62468] core_worker.cc:941: :info_message: Attempting to recover 2 lost objects by resubmitting their tasks or setting a new primary location from existing copies. To disable object reconstruction, set @ray.remote(max_retries=0).


: 

In [39]:
import gc

gc.collect()

0

In [ ]:
class LuminarEncoderRaid(LuminarEncoder):
    def __init__(self):
        super().__init__(256, "gpt2", key="generation")

### Encode Samples

In [41]:
tiny_ds = ray_ds.limit(2560)
tiny_ds

limit=2560
+- Dataset(
      num_rows=5615820,
      schema={
         id: string,
         adv_source_id: string,
         source_id: string,
         model: string,
         decoding: string,
         repetition_penalty: string,
         attack: string,
         domain: string,
         title: string,
         prompt: string,
         generation: string
      }
   )

In [47]:
ds = tiny_ds.map_batches(
    LuminarEncoderRaid,
    concurrency=10,
    batch_size=128,
)
ds.write_parquet("data/raid-gpt2-256-first.parquet", overwrite=True)

2025-05-13 19:21:35,508	INFO logging.py:290 -- Registered dataset logger for dataset dataset_6c9ecd80d7ad4139bb83e54952b64013_0
2025-05-13 19:21:35,508	INFO logging.py:298 -- dataset_6c9ecd80d7ad4139bb83e54952b64013_0 registers for logging while another dataset dataset_28_0 is also logging. For performance reasons, we will not log to the dataset dataset_6c9ecd80d7ad4139bb83e54952b64013_0 until it is the only active dataset.
2025-05-13 19:21:35,510	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_6c9ecd80d7ad4139bb83e54952b64013_0. Full logs are in /tmp/ray/session_2025-05-13_18-37-32_148385_1350620/logs/ray-data
2025-05-13 19:21:35,510	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_6c9ecd80d7ad4139bb83e54952b64013_0: InputDataBuffer[Input] -> LimitOperator[limit=2560] -> ActorPoolMapOperator[MapBatches(LuminarEncoderRaid)] -> TaskPoolMapOperator[Write]


Running 0: 0.00 row [00:00, ? row/s]

2025-05-13 19:21:40,434	ERROR exceptions.py:73 -- Exception occurred in Ray Data or Ray Core internal code. If you continue to see this error, please open an issue on the Ray project GitHub page with the full stack trace below: https://github.com/ray-project/ray/issues/new/choose
2025-05-13 19:21:40,434	ERROR exceptions.py:81 -- Full stack trace:
Traceback (most recent call last):
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/exceptions.py", line 49, in handle_trace
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/plan.py", line 510, in execute
    blocks = execute_to_legacy_block_list(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/legacy_compat.py", line 120, in execute_to_legacy_block_list
    bundles = executor.execute(dag, initial_stats=stats)

ActorDiedError: The actor died because of an error raised in its creation task, [36mray::_MapWorker.__init__()[39m (pid=1354939, ip=141.2.89.10, actor_id=b5e20e564e3653a2f198951804000000, repr=MapWorker(MapBatches(LuminarEncoderRaid)))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/staff_homes/mastoeck/python/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/operators/actor_pool_map_operator.py", line 425, in __init__
    self._map_transformer.init()
  File "/home/staff_homes/mastoeck/python/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/operators/map_transformer.py", line 203, in init
    self._init_fn()
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/planner/plan_udf_map_op.py", line 273, in init_fn
  File "/nvme/projects/PrismAI/PrismAI/.venv/lib/python3.12/site-packages/ray/data/_internal/execution/util.py", line 70, in __init__
  File "/tmp/ipykernel_1354780/4182275563.py", line 3, in __in

### Run Training

In [ ]:
model = LuminarClassifier()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from tqdm import tqdm


train_dataset = dataset["train"].with_format("torch", ["features", "label"])
for batch in tqdm(DataLoader(train_dataset, 32)):
    optimizer.zero_grad()
    features = batch["features"]
    labels = batch["label"].float().unsqueeze(-1)

    preds = model(features)

    loss = criterion(preds, labels)

    loss.backward()
    optimizer.step()

In [ ]:
import numpy as np
from sklearn.metrics import f1_score


y_pred, y_truth, losses = [], [], []
test_dataset = dataset["test"].with_format("torch", ["features", "label"])
for batch in tqdm(DataLoader(test_dataset, 32)):
    with torch.no_grad():
        features = batch["features"]
        labels = batch["label"].float().unsqueeze(-1)
        preds = model(features)

        y_pred.extend(preds.sigmoid().round().squeeze().tolist())
        y_truth.extend(labels.squeeze().tolist())

        loss = criterion(preds, labels)
        losses.append(loss.item())

print(f"loss={np.mean(losses)}")
print(f"f1={f1_score(y_truth, y_pred)}")